In [1]:
import pymgrid
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from random import sample, random

from pymgrid import MicrogridGenerator as mg


class Microgrid:

    def __init__(self, n_participants: int):
        self.participants = []
        self._current_t = 0

        env = mg.MicrogridGenerator(nb_microgrid=n_participants)
        env.generate_microgrid(verbose=True)

        self.participants = env.microgrids

    def get_current_step_obs(self, size_of_slot: int = 24): #to get the state tuple for fixed time-slot 

        d_t = 0 #total load demand
        sum_e_t = 0 #total consumption
        es_t = 0 #total surplus
        v_h = [] #list of load demands for every time slot

        for participant in self.participants:

            participant_consumption = participant._load_ts.iloc[self._current_t][0]
            participant_generation = participant._pv_ts.iloc[self._current_t][0]

            demand_variation = 0.05*participant_consumption if random() < 0.5 else -0.05*participant_consumption
            participant_demand = participant_consumption + demand_variation

            d_t += participant_demand
            sum_e_t += participant_consumption
            v_h.append(participant_consumption) #shouldn't we append demand not consumption?

            # Check surplus constraints
            surplus = participant_generation - participant_consumption
            es_t += surplus if surplus > 0 else 0

        # Compute the period of the day

        h_t = self._current_t % size_of_slot

        # Compute c_t

        v_h_t = np.mean(v_h)
        b_h = v_h_t*np.arange(0.25,2,0.25)
        b_h_t = sample(b_h)
        alpha_t = 0.02

        c_t = alpha_t * sum_e_t + b_h_t * sum_e_t**2

        return d_t, h_t, c_t, es_t

        # mg0 = env.microgrids[0]
        # mg1 = env.microgrids[1]
        # mg2 = env.microgrids[2]
        #
        # p_0 = self.participants
        #
        # mg0._load_ts[0:24*3].plot()
        #
        # plt.show()
        #
        # mg0._pv_ts[0:24*3].plot()
        #
        # plt.show()
        #
        # pd.DataFrame(mg0._pv_ts[0:24*3].to_numpy()-mg0._load_ts[0:24*3].to_numpy()).plot()



    # def loads(self):
    #
    #     for participant in self.participants:


if __name__ == '__main__':

    mgrid = microgrid(n_participants=1)


/home/madog/Software/anaconda3/envs/ml-707/lib/python3.9/site-packages/pymgrid/MicrogridGenerator.py:612: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,load,cost_loss_load,cost_overgeneration,cost_co2,PV_rated_power,battery_soc_0,battery_power_charge,battery_power_discharge,battery_capacity,battery_efficiency,battery_soc_min,battery_soc_max,battery_cost_cycle,grid_weak,grid_power_import,grid_power_export
0,960,10,1,0.1,969.6,0.2,430,430,1718,0.9,0.2,1,0.02,0,1920,1920
